# 第13章 モナドがいっぱい


```
fmap :: (Functor f) => (a -> b) -> f a -> f b

(<*>) :: (Applicative f) => f (a -> b) -> f a -> f b
```

In [1]:
(*) <$> Just 2 <*> Just 8

Just 16

In [2]:
(++) <$> Just "exdeath" <*> Nothing

Nothing

モナドはある願いを叶えるためのアプリカティブ値の自然な拡張  
「普通の値aを取って、文脈付きの値を返す関数に、文脈付きの値m aを渡したい」という願い  
つまり、a -> m b型の関数をm a型の値に適用したい。ということ

```
(>>=) :: (Monad a) => m a -> (a -> m b) -> m b
```
ここからは `f a` ではなく、`m a` と書くことにする  
`>>=` は bindと呼ばれる  

### Maybeから始めるモナド



In [3]:
fmap (++ "!") (Just "wisdom")

Just "wisdom!"

In [4]:
Just (+3) <*> Just 3

Just 6

`>>=` はモナド値と普通の値を取る関数を引数にとり、なんとかしてその関数をモナド地に適用してモナド値を得る  


In [5]:
(\x -> Just(x+1)) 1

Just 2

In [6]:
applyMaybe :: Maybe a -> (a -> Maybe b) -> Maybe b
applyMaybe Nothing f = Nothing
applyMaybe (Just x) f = f x


In [7]:
Just 3 `applyMaybe` \x -> Just (x + 1)

Just 4

In [8]:
Nothing `applyMaybe` \x -> Just (x + 1)

Nothing

In [9]:
Just 3 `applyMaybe` \x -> if x > 2 then Just x else Nothing

Just 3

In [10]:
Just 1 `applyMaybe` \x -> if x > 2 then Just x else Nothing

Nothing

### Monad型クラス

```
class Monad m where
    return :: a -> m a
    
    (>>=) :: m a -> (a -> m b) -> m b
    
    (>>) :: m a -> m b -> m b
    x >> y = x >> \_ -> y
    
    fail :: String -> m a
    fail msg = error msg
```

`class (Applicative m) = > Monad m where` という型クラス制約はないけど、全てのモナドはアプリカティブファンクター  

```
instance Monad Maybe where
    return x = Just x
    Nothing >>= f = Nothing
    Just x >>= f = f x
    fail _ = Nothing
```


In [13]:
return 1 :: Maybe Int

Just 1

次回

## 綱渡り

デュエルスタンバイ

In [1]:
type Birds = Int
type Pole = (Birds, Birds)

landLeft :: Birds -> Pole -> Pole
landLeft n (left, right) = (left + n, right)

landRight :: Birds -> Pole -> Pole
landRight n (left, right) = (left, n + right)


In [2]:
landLeft 2 (0, 0)

(2,0)

In [4]:
landLeft 2 (landLeft 1 (0,0))

(3,0)

In [5]:
x -: f = f x

In [6]:
100 -: (*3)

300

In [7]:
(0, 0) -: landLeft 2

(2,0)

In [8]:
(0, 0) -: landLeft 2 -: landRight 2 

(2,2)

In [18]:
-- 失敗をNothingで表現する

landLeft :: Birds -> Pole -> Maybe Pole
landLeft n (left, right) 
    |abs ((left + n) - right) < 4 = Just (left+n, right)
    | otherwise = Nothing
    
landRight :: Birds -> Pole -> Maybe Pole
landRight n (left, right)
    | abs (left - (right + n)) < 4 = Just (left, n + right)
    |otherwise = Nothing

In [12]:
landLeft 2 (0, 0)

Just (2,0)

In [13]:
landLeft 10 (0, 3)

Nothing

In [15]:
landRight 1 (0, 0) >>= landLeft 2

Just (2,1)

In [16]:
landRight 1 (0, 0) >>= landLeft 2 >>= landLeft 1

Just (3,1)

In [17]:
return (0, 0) >>= landRight 2 >>= landLeft 1 >>= landLeft 2

Line 1: Monad law, left identity
Found:
return (0, 0) >>= landRight 2
Why not:
landRight 2 (0, 0)

Just (3,2)

ばななで転ばせて落とす例ｗ

In [19]:
banana :: Pole -> Maybe Pole
banana _ = Nothing

In [21]:
return (0, 0) >>= landLeft 1 >>= banana >>= landRight 1

Line 1: Monad law, left identity
Found:
return (0, 0) >>= landLeft 1
Why not:
landLeft 1 (0, 0)

Nothing

### do記法

In [22]:
Just 3 >>= (\x -> Just (show x ++ "!"))

Just "3!"

In [23]:
Just 3 >>= (\x -> Just "!" >>= (\y -> Just (show x ++ y)))

Just "3!"

In [24]:
let x = 3; y ="!" in show x ++ y

"3!"

In [30]:
foo :: Maybe String
foo = Just 3 >>= (\x -> 
    Just "!" >>= (\y -> 
    Just (show x ++ y)))

In [31]:
foo

Just "3!"

In [32]:
-- doを使うとこう書ける

foo :: Maybe String
foo = do
    x <- Just 3
    y <- Just "!"
    Just (show x ++ y)

In [33]:
foo

Just "3!"

Scalaでいう flatMapが >>= で forがdoになるのか

In [34]:
routine :: Maybe Pole
routine = do
    start <- return (0, 0)
    first <- landLeft 2 start
    second <- landRight 2 first
    landLeft 1 second

Line 2: Use let
Found:
do start <- return (0, 0)
   first <- landLeft 2 start
   second <- landRight 2 first
   landLeft 1 second
Why not:
do let start = (0, 0)
   first <- landLeft 2 start
   second <- landRight 2 first
   landLeft 1 second

In [35]:
wopwop :: Maybe Char
wopwop = do
    (x: xs) <- Just ""
    return x

In [36]:
wopwop

Nothing

パターンマッチが失敗していてもNothingになる

### リストモナド

Monadインスタンス

```
instance Monad [ ] where
    return x = [x]
    xs >>= f = concat (map f xs)
    fail _ = []
```



In [37]:
[3,4,5] >>= \x -> [x,-x]

[3,-3,4,-4,5,-5]

In [38]:
[1,2] >>= \n -> ['a', 'b'] >>= \ch -> return (n, ch)

Line 1: Use string literal
Found:
['a', 'b']
Why not:
"ab"

[(1,'a'),(1,'b'),(2,'a'),(2,'b')]

In [39]:
listOfTuples :: [(Int, Char)]
listOfTuples = do
    n <- [1,2]
    ch <- ['a', 'b']
    return (n, ch)

Line 4: Use string literal
Found:
['a', 'b']
Why not:
"ab"

In [40]:
listOfTuples

[(1,'a'),(1,'b'),(2,'a'),(2,'b')]

In [41]:
[(n, ch)| n <- [1,2], ch <- ['a', 'b']]

Line 1: Use string literal
Found:
['a', 'b']
Why not:
"ab"

[(1,'a'),(1,'b'),(2,'a'),(2,'b')]

次回

## MonadPlusとguard関数

デュエルスタンバイ